In [37]:
%run init_notebookspace.py
from settings import POST_EST_DIR

DATA_DIR is existant under: C:\Users\LukasGrahl\Documents\GIT\memoire1\data


In [39]:
from src.utils import get_most_recent_mod_output
from src.bayesian import get_xarr_InferenceData

import xarray as xr
import numpy as np

import arviz as az
import pandas as pd

import os

In [49]:
import pickle
with open(os.path.join(DATA_DIR, f'mod6_nk_energy_lin2_y_pi_s_kalman_forecast.pkl'), 'rb') as f:
    rbc_dict = pickle.load(f)
with open(os.path.join(DATA_DIR, f'bvar_forecast.pkl'), 'rb') as f:
    bvar_dict = pickle.load(f)

In [3]:
import os

In [4]:
def load_trace_data(file_path: str):
    if 'mod' in file_path:
        return xr.open_dataset(file_path)
    elif 'bvar' in file_path:
        return az.from_netcdf(file_path)
    else:
        raise ValueError('Unknown file tpye')

In [14]:
from config import mod4_params, mod4_priors, mod5_params, mod5_priors, mod6_params, mod6_priors
mods = {
    'mod4_rbc_vanilla': {'is_mod': True},
    # 'mod5_nk_vanilla': {'is_mod': True},
    # 'mod6_nk_energy_lin2': {'is_mod': True},
    'bvar': {'is_mod': False},
}

# load model
for key in mods.keys():
    file_name = get_most_recent_mod_output(POST_EST_DIR, f'{key}_accepted')
    print(file_name)
    mods[key]['post_out'] = load_trace_data(os.path.join(POST_EST_DIR, file_name))

mod4_rbc_vanilla_accepted_20230523_0902.nc
bvar_accepted_20230523_1409.nc


In [15]:
def get_xarr_InferenceData(xarr_in: xr.Dataset):
    # xarr = xarr_in.where(xarr_in.draw >= 8996).dropna('draw').copy()
    draw = len(xarr.draw)

    xr_prior = xr.Dataset(
        {
            key: (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0] * draw).reshape(draw, 1),
                        np.array(xarr.sel(parameter=key).prior_param).reshape(draw, 1),
                        np.array([0] * draw).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
            for key in xarr.parameter.values
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )

    xr_post = xr.Dataset(
        {
            key: (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0] * draw, dtype=float).reshape(draw, 1),
                        np.array(xarr.sel(parameter=key).posterior_param, dtype=float).reshape(draw, 1),
                        np.array([0] * draw, dtype=float).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
            for key in xarr.parameter.values
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )

    xr_loglike = xr.Dataset(
        {
            'obs': (
                ['chain', 'draw', 'a_dim'],
                np.concatenate(
                    [
                        np.array([0.] * draw, dtype=float).reshape(draw, 1),
                        np.array(xarr.log_likelihood, dtype=float).reshape(draw, 1),
                        np.array([0.] * draw, dtype=float).reshape(draw, 1)
                    ],
                    axis=1
                ).reshape(1, draw, 3)
            )
        },
        coords={
            "chain": (["chain"], np.arange(1)),
            "draw": (["draw"], np.arange(draw)),
            "a_dim": (["a_dim"], ["x", "y", "z"])

        }
    )
    return xr_prior, xr_post, xr_loglike

In [16]:
for key in mods.keys():
    xarr = mods[key]['post_out']  
    if mods[key]['is_mod']:
        mods[key]['az_inf'] = az.InferenceData(**dict(zip(
            ['prior', 'posterior', 'log_likelihood'], 
            *[get_xarr_InferenceData(xarr)]
        )))
    else:
        mods[key]['az_inf'] = mods[key]['post_out']

In [56]:
z = az.waic(mods['bvar']['az_inf'])
z

C:\Users\LukasGrahl\miniconda3\envs\env_conda3_memoire1\lib\site-packages\arviz\stats\stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 8000 posterior samples and 335 observations log-likelihood matrix.

          Estimate       SE
elpd_waic  1177.71    63.43
p_waic      104.69        -

There has been a warning during the calculation. Please check the results.

In [58]:
x = az.waic(mods['mod4_rbc_vanilla']['az_inf'])
x

C:\Users\LukasGrahl\miniconda3\envs\env_conda3_memoire1\lib\site-packages\arviz\stats\stats.py:1645: UserWarning: For one or more samples the posterior variance of the log predictive densities exceeds 0.4. This could be indication of WAIC starting to fail. 
See http://arxiv.org/abs/1507.04544 for details
  warnings.warn(


Computed from 696 posterior samples and 3 observations log-likelihood matrix.

          Estimate       SE
elpd_waic -703453212053.18  574367142484.05
p_waic    703452633541.18        -

There has been a warning during the calculation. Please check the results.